# FSGP 2024 SOC from voltage using cell datasheet

Using the functions in `soc_from_voltage.py`, we can now estimate the state of charge at points during FSGP 2024. This method for estimating SOC is relatively crude since it doesn't account for the way the battery reacts to being charged or discharged at different rates. This method works best if the datasheet voltage is accurate, (and that we are close to the 2A curve provided), and when the battery is in a 'relaxed' state, so we can read its equilibrium voltage.

To use the cell data to estimate the SOC during the race, we need to get a best guess as to what the relaxed battery voltage would be. To do this, I take a windowed maximum of the voltage across the battery, which is likely to represent a point in time where there is not much current being drawn. This is still not very accurate because in reality the battery needs more time to relax. The most relevant real application of this method for estimating SOC is to get a quick estimate of how much energy remains during race down-time, such as at the end of the race day.

In [1]:
from data_tools import InfluxClient

from soc_from_voltage import (cell_soc_from_voltage,
                              cell_wh_from_voltage,
                              MAX_CELL_ENERGY,
                              MIN_CELL_VOLTAGE,
                              MAX_CELL_VOLTAGE)

import numpy as np
import matplotlib.pyplot as plt
from data_tools.fsgp_2024_laps import FSGPDayLaps

laps_3 = FSGPDayLaps(3)
day_3_start = laps_3.get_start_utc(1)
day_3_stop = laps_3.get_finish_utc(laps_3.get_lap_count())

client = InfluxClient()

day_3_minimum_module = client.query_time_series(day_3_start, day_3_stop, "VoltageOfLeast")
day_3_pack_voltage = client.query_time_series(day_3_start, day_3_stop, "TotalPackVoltage")

plt.plot(day_3_minimum_module)
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.title('Day 3, Min. Module Voltage')
plt.show()

plt.plot(day_3_pack_voltage)
plt.xlabel('Time')
plt.ylabel('Voltage')
plt.title('Day 3, Total Pack Voltage')
plt.show()

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.27.1', 'Date': 'Sat, 12 Oct 2024 03:47:29 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '55', 'Connection': 'keep-alive', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.1', 'X-Platform-Error-Code': 'unauthorized'})
HTTP response body: b'{"code":"unauthorized","message":"unauthorized access"}'
